# 01 Baseline from scratch
Hypothesis: with low target labels, scratch training converges slower than transfer-based methods.

### Expected Outcome
Scratch should define the lowest stable baseline for later comparisons.

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
while ROOT != ROOT.parent and not (ROOT / 'src').is_dir():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT / 'src'))

from utils.notebook_transfer import TransferNotebookLab

LAB = TransferNotebookLab.from_root(ROOT)
FAST_DEV_RUN = False

In [ ]:
LAB.run(
    base_name='transfer_core_related.yaml',
    notebook_tag='01_scratch',
    fast_dev_run=FAST_DEV_RUN,
    overrides={
        'methods': ['scratch'],
        'train': {'source_epochs': 4, 'target_epochs': 12},
    },
)
scratch = LAB.read_method('scratch')
scratch[['epoch', 'target_test_acc', 'feature_drift']].tail()

In [ ]:
fig, ax = plt.subplots(figsize=(6.2, 3.6))
ax.plot(scratch['epoch'], scratch['target_test_acc'], marker='o', label='scratch')
ax.set_title('Scratch target accuracy')
ax.set_xlabel('epoch')
ax.set_ylabel('target_test_acc')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '01_scratch_target_acc.png')

fig, ax = plt.subplots(figsize=(6.2, 3.6))
ax.plot(scratch['epoch'], scratch['feature_drift'], marker='o', color='#E45756')
ax.set_title('Scratch feature drift')
ax.set_xlabel('epoch')
ax.set_ylabel('feature_drift')
ax.grid(alpha=0.25)
LAB.savefig(fig, '01_scratch_feature_drift.png')

### Interpretation
Use this as the denominator for all transfer claims.